In [47]:
folder_path = "../Twitter_Data/"
cn_folder_path = "../Twitter_Data/Twitter_Data_Chinese/"

visuals_output_path = "./visuals/"
output_path = "./output/"

file1 = "Twitter_Covid-19_Lockdown_5000.csv"
file2 = "Twitter_Jan_Mar_5000.csv"
file3 = "Twitter_Mar_5000.csv"
file4 = "Twitter_May_Nov_5000.csv"

cn_file1 = "en.Twitter_Covid-19_Lockdown_5000_chinese.csv"
cn_file2 = "en.Twitter_Jan_Apr_2020_5000_chinese.csv"
cn_file3 = "en.Twitter_May_June_2022_5000_chinese.csv"

In [48]:
!pip install pyLDAvis

In [49]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

import nltk
nltk.download('stopwords') #download if don't have yet
from nltk.tokenize import word_tokenize, RegexpTokenizer # tokenize words
from nltk.corpus import stopwords

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lindy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [50]:
# Read in files
df1 = pd.read_csv(folder_path + file1)
df2 = pd.read_csv(folder_path + file2)
df3 = pd.read_csv(folder_path + file3)
df4 = pd.read_csv(folder_path + file3)

cn_df1 = pd.read_csv(cn_folder_path + cn_file1)
cn_df2 = pd.read_csv(cn_folder_path + cn_file2)
cn_df3 = pd.read_csv(cn_folder_path + cn_file3)

<h1>Cleaning Data</h1>

In [51]:
# Drop columns mentioning Bheed trailer
df1 = df1[df1["text"].str.contains("Bheed")==False]
df2 = df2[df2["Text"].str.contains("Bheed")==False]
df3 = df3[df3["Text"].str.contains("Bheed")==False]
df4 = df4[df4["Text"].str.contains("Bheed")==False]

In [52]:
list1 = df1["text"].tolist()
list2 = df2["Text"].tolist()
list3 = df3["Text"].tolist()
list4 = df4["Text"].tolist()

cn_list1 = cn_df1["text"].tolist()
cn_list2 = cn_df2["text"].tolist()
cn_list3 = cn_df3["text"].tolist()

In [53]:
def remove_user_mentions(text):
    return re.sub("@[A-Za-z0-9_]+","", text)

def remove_links(text):
    return re.sub(r'http\S+', '', text)

def remove_digit_strings(text):
    return re.sub(r'\d+', '', text)

def remove_special_chars(text):
    remove_chars = '[0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+'
    return re.sub(remove_chars, ' ', text)

def clean_text(text):
    result_text = text
    result_text = remove_user_mentions(result_text)
    result_text = remove_links(result_text)
    result_text = remove_digit_strings(result_text)
    result_text = remove_special_chars(result_text)
    result_text = result_text.lower()
    return result_text

def remove_cn_chars(text):
    result_text = re.sub(r'([\u4e00-\u9fff]+', '', text)
    return result_text

In [54]:
for i in range(len(list1)):
    list1[i] = clean_text(list1[i])

for i in range(len(list2)):
    list2[i] = clean_text(list2[i])

for i in range(len(list3)):
    list3[i] = clean_text(list3[i])
    
for i in range(len(list4)):
    list4[i] = clean_text(list4[i])
    
for i in range(len(cn_list1)):
#     cn_list1[i] = remove_cn_chars(cn_list1[i])
    cn_list1[i] = clean_text(cn_list1[i])

# for i in range(len(cn_list2)):
#     cn_list2[i] = remove_cn_chars(cn_list2[i])
#     cn_list2[i] = clean_text(cn_list2[i])

# for i in range(len(cn_list3)):
#     cn_list3[i] = remove_cn_chars(cn_list3[i])
#     cn_list3[i] = clean_text(cn_list3[i])

In [57]:
df1["cleaned_text"] = list1
df2["cleaned_text"] = list2
df3["cleaned_text"] = list3
df4["cleaned_text"] = list4

cn_df1["cleaned_text"] = cn_list1
cn_df2["cleaned_text"] = cn_list2
cn_df3["cleaned_text"] = cn_list3

In [58]:
# Create dictionary of period of tweets to information associated with tweet

tweets = {}
keys = ["covid_lockdown", "jan_mar", "mar", "may_nov", "cn_covid_lockdown"]
for key in keys:
    tweets[key] = {}

tweets["covid_lockdown"]["text"] = df1["cleaned_text"].tolist()
tweets["jan_mar"]["text"] = df2["cleaned_text"].tolist()
tweets["mar"]["text"] = df3["cleaned_text"].tolist()
tweets["may_nov"]["text"] = df4["cleaned_text"].tolist()

tweets["cn_covid_lockdown"]["text"] = cn_df1["cleaned_text"].tolist()
# tweets["cn_jan_apr_2020"]["text"] = cn_df2["cleaned_text"].tolist()
# tweets["cn_may_june_2022"]["text"] = cn_df3["cleaned_text"].tolist()

<h1>Tokenizing Data</h1>

In [59]:
# Get custom stopwords
content = []
f = open("./stopwords.txt", encoding = 'utf-8')
# perform file operations
for line in f:
    content.append(line)
f.close()

custom_stopwords = []
for line in content:
    wordlist = line.split(",")
    for word in wordlist:
        custom_stopwords.append(word)

In [60]:
# Stopwords
stop_words = stopwords.words('english')
exclude_words = stop_words

exclude_words_extra = ["RT","still","covid","coronavirus","lockdown","lockdo","pandemic","let","get","ago","go","im","ive","would","one","also","to","tag"]

# Exclude custom stopwords
exclude_words.extend(custom_stopwords)
exclude_words.extend(exclude_words_extra)

In [61]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

for period in tweets.keys():
    tweets[period]["data_words"] = list(sent_to_words(tweets[period]["text"]))

In [62]:
def bigram_trigram_models(period):
    data_words = tweets[period]["data_words"]
    
    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
    
    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

    tweets[period]["bigram"] = bigram
    tweets[period]["trigram"] = trigram
    tweets[period]["bigram_mod"] = bigram_mod
    tweets[period]["trigram_mod"] = trigram_mod

for period in tweets.keys():
    bigram_trigram_models(period)

In [63]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in exclude_words] for doc in texts]

def make_bigrams(texts, bigram_mod):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts, bigram_mod, trigram_mod):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, nlp, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [64]:
def combined(period):
    data_words = tweets[period]["data_words"]

    # Remove Stop Words
    data_words_nostops = remove_stopwords(data_words)
    data_words_nostops = data_words
    tweets[period]["data_words_nostops"] = data_words_nostops 

    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words_nostops, tweets[period]["bigram_mod"])
    tweets[period]["data_words_bigrams"] = data_words_bigrams

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized = lemmatization(data_words_bigrams, nlp, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    tweets[period]["data_lemmatized"] = data_lemmatized

for period in tweets.keys():
    combined(period)

In [65]:
print(tweets[list(tweets.keys())[0]].keys())

dict_keys(['text', 'data_words', 'bigram', 'trigram', 'bigram_mod', 'trigram_mod', 'data_words_nostops', 'data_words_bigrams', 'data_lemmatized'])


In [66]:
def tokenize(period):
    data_lemmatized = tweets[period]["data_lemmatized"]

    # Create Dictionary
    id2word = corpora.Dictionary(data_lemmatized)
    tweets[period]["id2word"] = id2word

    # Create Corpus
    texts = data_lemmatized
    tweets[period]["texts"] = texts

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]
    tweets[period]["corpus"] = corpus

    # Human readable format of corpus (term-frequency)
    tweets[period]["corpus_readable"] = [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

for period in tweets.keys():
    tokenize(period)

In [67]:
index = 1   # change this to see diff output
print(tweets[list(tweets.keys())[index]]["corpus_readable"])

[[('covid', 1), ('home', 1), ('lockdown', 1), ('place', 1)]]


<h1>LDA Model</h1>

In [68]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

# supporting function 2
def compute_perplexity_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    perplexity_score = lda_model.log_perplexity(corpus_sets[i])
    
    return perplexity_score

In [69]:
def lda_model(period):
    corpus = tweets[period]["corpus"]
    id2word = tweets[period]["id2word"]

    # Build LDA model
    num_topics = 4
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=num_topics, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

    tweets[period]["lda_model"] = lda_model

for period in tweets.keys():
    lda_model(period)

In [70]:
index = 1   # change this to see diff output
print(tweets[list(tweets.keys())[index]]["lda_model"])

LdaModel<num_terms=3084, num_topics=4, decay=0.5, chunksize=100>


In [71]:
def keyword_in_topics(period):
    corpus = tweets[period]["corpus"]
    lda_model = tweets[period]["lda_model"]

    # Print the Keyword in the 10 topics
    pprint(lda_model.print_topics())
    doc_lda = lda_model[corpus]
    tweets[period]["doc_lda"] = doc_lda

for period in tweets.keys():
    keyword_in_topics(period)

[(0,
  '0.103*"lockdown" + 0.082*"covid" + 0.058*"year" + 0.047*"first" + '
  '0.029*"support" + 0.026*"today" + 0.021*"anniversary" + 0.020*"mark" + '
  '0.016*"third" + 0.015*"pandemic"'),
 (1,
  '0.055*"lockdown" + 0.053*"covid" + 0.019*"amp" + 0.017*"more" + '
  '0.016*"impact" + 0.012*"mask" + 0.011*"wear" + 0.011*"inequality" + '
  '0.010*"expenditure_data" + 0.010*"collected_by"'),
 (2,
  '0.036*"covid" + 0.030*"lockdown" + 0.026*"people" + 0.026*"day" + '
  '0.024*"make" + 0.022*"happen" + 0.021*"post" + 0.020*"mental" + '
  '0.019*"population" + 0.017*"hlth"'),
 (3,
  '0.055*"covid" + 0.054*"lockdown" + 0.028*"former" + 0.024*"party" + '
  '0.021*"boris_johnson" + 0.020*"truth" + 0.019*"speak" + 0.017*"shoot" + '
  '0.017*"sheep" + 0.017*"agenda_klausschw"')]
[(0,
  '0.031*"town" + 0.013*"say" + 0.013*"base" + 0.012*"order" + 0.010*"close" + '
  '0.010*"ban" + 0.010*"video" + 0.009*"public" + 0.008*"escape" + '
  '0.008*"datum"'),
 (1,
  '0.023*"chinese" + 0.019*"start" + 0.01

In [72]:
index = 1   # change this to see diff output
print(tweets[list(tweets.keys())[index]]["doc_lda"])

In [73]:
def perplexity_coherence(period):
    corpus = tweets[period]["corpus"]
    data_lemmatized = tweets[period]["data_lemmatized"]
    id2word = tweets[period]["id2word"]
    lda_model = tweets[period]["lda_model"]

    # Compute Perplexity
    tweets[period]["perplexity"] = lda_model.log_perplexity(corpus)
    # print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()

    tweets[period]["coherence_model_lda"] = coherence_model_lda
    tweets[period]["coherence_lda"] = coherence_lda

    # print('\nCoherence Score: ', coherence_lda)

for period in tweets.keys():
    perplexity_coherence(period)

In [74]:
index = 1   # change this to see diff output
print(tweets[list(tweets.keys())[index]]["perplexity"])
print(tweets[list(tweets.keys())[index]]["coherence_lda"])

-7.136573797604998
0.42403609035375367


In [75]:
def optimal_model(period): 
    lda_model = tweets[period]["lda_model"]

    optimal_model = lda_model
    model_topics = optimal_model.show_topics(formatted=False)

    tweets[period]["model_topics"] = model_topics

for period in tweets.keys():
    optimal_model(period)

In [76]:
index = 1   # change this to see diff output
print(tweets[list(tweets.keys())[index]]["model_topics"])

[(0, [('town', 0.031279), ('say', 0.01269617), ('base', 0.0125529915), ('order', 0.0123109035), ('close', 0.010081722), ('ban', 0.009998816), ('video', 0.009547917), ('public', 0.008669036), ('escape', 0.008173242), ('datum', 0.008055627)]), (1, [('chinese', 0.022847172), ('start', 0.018699147), ('official', 0.015295484), ('see', 0.014387247), ('safe', 0.013070171), ('all', 0.012088214), ('include', 0.011151277), ('run', 0.0093834), ('million', 0.009127393), ('let', 0.009045857)]), (2, [('case', 0.034101836), ('lockdown', 0.02500201), ('day', 0.020175047), ('covid', 0.020119369), ('people', 0.01776956), ('new', 0.015229243), ('life', 0.013526615), ('report', 0.012683687), ('coronavirus', 0.012562215), ('first', 0.011416238)]), (3, [('lockdown', 0.11147557), ('covid', 0.0969804), ('coronavirus', 0.04255527), ('city', 0.019650945), ('people', 0.01765093), ('ncov', 0.013701352), ('quarantine', 0.012985469), ('outbreak', 0.01220372), ('day', 0.01149202), ('now', 0.011431002)])]


In [77]:
def explore_topic(lda_model, topic_number, topn, output=True):
    """
    accept a ldamodel, atopic number and topn vocabs of interest
    prints a formatted list of the topn terms
    """
    terms = []
    for term, frequency in lda_model.show_topic(topic_number, topn=topn):
        terms += [term]
        if output:
            print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))
    
    return terms

In [80]:
def explore_topic_by_title(period):
    lda_model = tweets[period]["lda_model"]
    topic_summaries = []

    print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')
    print(period)
    for i in range(4):
        print('Topic '+str(i)+' |---------------------\n')
        tmp = explore_topic(lda_model,topic_number=i, topn=10, output=True )
        #     print tmp[:5]
        topic_summaries += [tmp[:5]]
        # print

    tweets[period]["topic_summaries"] = topic_summaries

for period in tweets.keys():
    explore_topic_by_title(period)

term                 frequency

covid_lockdown
Topic 0 |---------------------

lockdown             0.103
covid                0.082
year                 0.058
first                0.047
support              0.029
today                0.026
anniversary          0.021
mark                 0.020
third                0.016
pandemic             0.015
Topic 1 |---------------------

lockdown             0.055
covid                0.053
amp                  0.019
more                 0.017
impact               0.016
mask                 0.012
wear                 0.011
inequality           0.011
expenditure_data     0.010
collected_by         0.010
Topic 2 |---------------------

covid                0.036
lockdown             0.030
people               0.026
day                  0.026
make                 0.024
happen               0.022
post                 0.021
mental               0.020
population           0.019
hlth                 0.017
Topic 3 |---------------------

covid          

<h1>Data Output - Dominant Topic</h1>

In [81]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

for period in tweets.keys():
    lda_model = tweets[period]["lda_model"]
    corpus = tweets[period]["corpus"]
    texts = tweets[period]["texts"]
    
    df_topic_sents_keywords = format_topics_sentences(lda_model, corpus, texts)
    
    # Format
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
    
    tweets[period]["df_dominant_topic"] = df_dominant_topic
    
    df_dominant_topic.to_csv(output_path + period + ".csv")

<h1>Data Visualization</h1>

<h3>PyLDAVis</h3>

In [82]:
import pyLDAvis.gensim_models as gensimvis
import pickle
import string

# Visualize the topics
for period in tweets.keys():
    lda_model = tweets[period]["lda_model"]
    corpus = tweets[period]["corpus"]
    id2word = tweets[period]["id2word"]

    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
    
    file_path = visuals_output_path + period + '.html'
    pyLDAvis.save_html(LDAvis_prepared, file_path)
    
    with open(file_path, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write("<h1>" + period + "</h1>" + '\n' + content)
        f.close()

C:\Users\Lindy\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
